#pycaretはtrain sampleとtest sampleを予め分けて準備し、4つのステップでtest sampleでの精度を解析する

①training dataとtest dataを読み込む

②狭義のtraining set (training sample全体の7割)を10分割交差検証で比較しハイパーパラメーターのチューニングする

③validation set (training sample全体の3割)も使用して、training sample全体でのモデルを作成する

④training set全体での精度の検証 (internal validation

⑤予め別に自身で準備したtest sampleで精度を検証する (external validation)
→この精度を上げるのが目的


In [ ]:
!pip install pycaret[full]

In [ ]:
#ライブラリのインポート
import numpy as np
import pandas as pd

In [ ]:
from pycaret.classification import *

#ここからは実際のデータの読み込み

In [ ]:
# ローカルファイルを直接アップロードする (train data)
from google.colab import files
uploaded = files.upload()


In [ ]:
#train dataの読み込み
train_data = pd.read_csv('train.csv')
train_data.head()

In [ ]:
# ローカルファイルを直接アップロードする (test data)
from google.colab import files
uploaded = files.upload()

In [ ]:
#test dataの読み込み
test_data = pd.read_csv('test.csv')
test_data.head()

#ここからステップ①、pycaretにデータの型を判別させ、training data全体ををtrainingのための狭義のtraining sampleとパラメーターを決めるためのvalidation sampleに分割する。


In [ ]:
#train_sizeを指定しない場合は7:3に分割される
#pycaretにデータの型を判別させる (sampleIDは無視する→ignore)
exp1 = setup(train_data, target = 'event', train_size = 0.8,ignore_features = ['sampleID'])

#ここからステップ② 狭義のtraining set (上でtraining sample全体の8割に指定)を10分割交差検証で比較しハイパーパラメーターのチューニングする

In [ ]:
#transformed train sampleでのmodelの比較 (デフォルトは10分割交差検証)
compare_models()

In [ ]:
#Logistic regressionモデルを作成
lr = create_model('lr')

In [ ]:
#勾配ブースティング決定木 (ブースティング)モデルを作成
xgboost = create_model('gbc')

In [ ]:
#ランダムフォレストモデルを作成
rf = create_model('rf')

In [ ]:
#サポートベクターマシン (SVM)を作成
rbfsvm = create_model('rbfsvm')

In [ ]:
#k近傍法 (knn)を作成
knn = create_model('knn')

In [ ]:
#Naive Bayesモデルを作成
nb = create_model('nb')

In [ ]:
#多層パーセプトロン (ニューラルネット)モデルを作成
mlp = create_model('mlp')

ここからハイパーパラメーターをチューニング、pycaretはランダムサーチで最適パラメーターを探索

In [ ]:
#試行回数の指定 n_iter = 10 (時間がかかるので10にしています、50や100に設定してください)
#AUCがいいものを選ぶ場合はoptimizeを'AUC'で指定、正答率なら'Accuracy'

#Logistic regression
tuned_lr = tune_model(lr,  n_iter = 10, fold=10,optimize = 'AUC')

In [ ]:
#ブースティング
tuned_xgboost = tune_model(xgboost,  n_iter = 10, fold=10,optimize = 'AUC')

In [ ]:
#ランダムフォレスト
tuned_rf = tune_model(rf,  n_iter = 10, fold=10,optimize = 'AUC')

In [ ]:
#サポートベクターマシン
tuned_rbfsvm = tune_model(rbfsvm,  n_iter = 10, fold=10,optimize = 'AUC')

In [ ]:
#k近傍法 (knn)
tuned_knn = tune_model(knn,  n_iter = 10, fold=10,optimize = 'AUC')

In [ ]:
#Naive beyesモデル
tuned_nb = tune_model(nb,  n_iter = 10, fold=10,optimize = 'AUC')

In [ ]:
#多層パーセプトロン (ニューラルネット)モデル
#mlpのチューニングは時々エラーになる (n_iterを減らすなどで対応)
tuned_mlp = tune_model(mlp,  n_iter = 10, fold=10,optimize = 'AUC')

#ここからステップ③ validation sampleも含めたtraining sample2100例全体でのモデル (final model)を固定する

In [ ]:
#ロジスティックモデルの固定
final_lr = finalize_model(tuned_lr)

In [ ]:
#ブースティングモデルの固定
final_xgboost = finalize_model(tuned_xgboost)

In [ ]:
#ランダムフォレストモデルの固定
final_rf = finalize_model(tuned_rf)

In [ ]:
#SVMモデルの固定
final_rbfsvm  = finalize_model(tuned_rbfsvm )

In [ ]:
#KNNモデルの固定
final_knn = finalize_model(tuned_knn)

In [ ]:
#Naive bayesモデルの固定
final_nb = finalize_model(tuned_nb)

In [ ]:
#mlp (ニューラルネットワーク)モデルの固定
final_mlp = finalize_model(tuned_mlp)

#ここからステップ④　training set全体での精度検証 (internal validation)

このコードは厳密にはtraining sample全体で評価を行っているのではなく、training sampleの一部である (今回ならtraining sampleの2割)　validation sampleのみでの評価を行っている

→training sample全体で行うためには、本プロセスの最後で行っているtest dataでの評価の際に用いるtest dataをtraining dataに置き換えて、test dataに行うのと同様にScoreのCSVファイルを出力する

In [ ]:
#ロジスティックモデルのinternal validation
predict_model(final_lr);

In [ ]:
#ブースティングモデルのinternal validation
predict_model(final_xgboost);

In [ ]:
#ランダムフォレストモデルのinternal validation
predict_model(final_rf);

In [ ]:
#SVMモデルのinternal validation
predict_model(final_rbfsvm);

In [ ]:
#knnモデルのinternal validation
predict_model(final_knn);

In [ ]:
#Naive bayesモデルのinternal validation
predict_model(final_nb);

In [ ]:
#MLP (ニューラルネットワーク)モデルのinternal validation
predict_model(final_mlp);

#ここからステップ⑤　予め別に自身で準備したtest sampleで精度を検証する (external validation)→この精度を上げるのが目的

In [ ]:
#ロジスティックモデルの評価
prediction_lr = predict_model(final_lr, data = test_data)
prediction_lr

In [ ]:
#ブースティングモデルの評価
prediction_xg = predict_model(final_xgboost, data = test_data)
prediction_xg

In [ ]:
#ランダムフォレストモデルの評価
prediction_rf = predict_model(final_rf, data = test_data)
prediction_rf

In [ ]:
#SVMモデルの評価
prediction_svm = predict_model(final_rbfsvm, data = test_data)
prediction_svm

In [ ]:
#knnモデルの評価
prediction_knn = predict_model(final_knn, data = test_data)
prediction_knn

In [ ]:
#Naive bayesモデルの評価
prediction_nb = predict_model(final_nb, data = test_data)
prediction_nb

In [ ]:
#MLPモデルの評価
prediction_mlp = predict_model(mlp, data = test_data)
prediction_mlp

#既存のスコアリングシステムなどとの比較をしやすくするため、統計ソフトでの解析用に機械学習が出したスコアと正解ラベルをcsvファイルで出力する。

注意点: AUCを出すためにはprediction_label=0のものは、prediction=1の確率に変換をする

In [ ]:
#ロジスティックモデルの予測出力 (左のフォルダにcsvが出ているので、ファイル右の点3つからダウンロード)
prediction_lr.to_csv('prediction_lr.csv', columns=['event','prediction_label','prediction_score'])

In [ ]:
#ブースティングの予測出力
prediction_xg.to_csv('prediction_xg.csv', columns=['event','prediction_label','prediction_score'])

In [ ]:
#ランダムフォレストの予測出力
prediction_rf.to_csv('prediction_rf.csv', columns=['event','prediction_label','prediction_score'])

In [ ]:
#SVMの予測出力
prediction_svm.to_csv('prediction_svm.csv', columns=['event','prediction_label','prediction_score'])

In [ ]:
#knnの予測出力
prediction_knn.to_csv('prediction_knn.csv', columns=['event','prediction_label','prediction_score'])

In [ ]:
#Naive bayesの予測出力
prediction_nb.to_csv('prediction_nb.csv', columns=['event','prediction_label','prediction_score'])

In [ ]:
#MLP (ニューラルネットワーク)の予測出力
prediction_mlp.to_csv('prediction_mlp.csv', columns=['event','prediction_label','prediction_score'])

#ハイパーパラメーターや変数の重要度等を一斉に出力する(catboostなどでは出力されない)

In [ ]:
evaluate_model(final_lr)

In [ ]:
evaluate_model (final_xgboost)

In [ ]:
evaluate_model (final_rf)

In [ ]:
evaluate_model (final_rbfsvm)

In [ ]:
evaluate_model (final_knn)

In [ ]:
evaluate_model (final_nb)

#ここから、アンサンブルでスタッキングモデルを作成する

In [ ]:
# スタッキングモデルを作成（メタ学習器にLogistic Regressionを使用）
stacked_model = stack_models(
    estimator_list=[final_lr, final_rf, final_xgboost],
    meta_model=create_model('lr'),  # メタ学習器
    fold=5  # クロスバリデーションのフォールド数
)

In [ ]:
# ランダムサーチでハイパーパラメーターチューニング
tuned_stacked = tune_model(
    stacked_model,
    optimize='AUC',                    # AUCで最適化
    search_library='scikit-learn',     # scikit-learnのRandomizedSearchCVを使用
    search_algorithm='random',         # ランダムサーチ
    n_iter=3,                        # 試行回数
    fold=5,                           # クロスバリデーションのフォールド数
    verbose=True
)

In [ ]:
# 全データで最終モデルを訓練
final_stacked_model = finalize_model(tuned_stacked)

In [ ]:
# 訓練データでの予測と評価
train_predictions = predict_model(final_stacked_model, data=train_data)

In [ ]:
# テストデータでの予測
test_predictions = predict_model(final_stacked_model, data=test_data)

# テストデータの実際の精度計算
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 予測結果の取得
y_true_test = test_data['event']
y_pred_test = test_predictions['prediction_label']
y_pred_proba_test = test_predictions['prediction_score']

In [ ]:
evaluate_model (final_stacked_model)